In [1]:
!pip install -U -q scikit-learn autogluon

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.8.2 requires keras-core, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
aiobotocore 2.12.1 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.29.165 which is incompatible.
albumentations 1.4.0 requires scikit-image>=0.21.0, but you have scikit-image 0.20.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.5 which is incomp

In [2]:
import pandas as pd
import numpy as np
import warnings
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_log_error
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularPredictor,TabularDataset

warnings.filterwarnings('ignore')
seed = 27

In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv', index_col='id')

In [4]:
train['Sex'] = train['Sex'].map({'M': 0, 'F': 1, 'I': 2})
test['Sex'] = test['Sex'].map({'M': 0, 'F': 1, 'I': 2})

In [5]:
_train, _val = train_test_split(train, test_size=0.1, random_state=seed)

In [6]:
_train = TabularDataset(_train)
_val = TabularDataset(_val)
test = TabularDataset(test)

In [7]:
scorer = make_scorer(
    name='root_mean_squared_log_error',
    score_func=root_mean_squared_log_error,
    greater_is_better=False,
    optimum=0
)

In [8]:
predictor = TabularPredictor(
    label="Rings",
    eval_metric=scorer,
    problem_type="regression",
    verbosity=2
).fit(
    train_data=_train,
    presets="best_quality",
    time_limit=3600 * 8
)

No path specified. Models will be saved in: "AutogluonModels/ag-20240404_064458"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 28800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240404_064458/ds_sub_fit/sub_fit_ho.
2024-04-04 06:45:00,016	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Beginning A

In [9]:
score = -predictor.evaluate(_val, silent=True)['root_mean_squared_log_error']

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [10]:
print(f"Validation RMSLE Score: {score}")

Validation RMSLE Score: 0.1481309790381286


### Why change the sign?
- Take a look at this issue on GitHub: https://github.com/autogluon/autogluon/issues/524

In [11]:
lb = predictor.leaderboard(_val, silent=True)

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [12]:
lb['score_test'] = lb['score_test'].apply(lambda x: -x)
lb['score_val'] = lb['score_val'].apply(lambda x: -x)

lb.style.background_gradient(subset=['score_val', 'score_test'], cmap='coolwarm')

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.148131,0.147773,root_mean_squared_log_error,152.474992,1450.057659,17943.786128,0.010069,0.006130,14.044604,3,True,60
1,NeuralNetFastAI_r191_BAG_L2,0.148464,0.148289,root_mean_squared_log_error,142.826374,1404.151929,15238.093719,3.463322,5.007308,1305.098559,2,True,50
2,NeuralNetTorch_r22_BAG_L2,0.148537,0.148639,root_mean_squared_log_error,139.974178,1400.824878,14718.175975,0.611126,1.680257,785.180814,2,True,53
3,NeuralNetFastAI_r102_BAG_L2,0.148583,0.148294,root_mean_squared_log_error,140.017187,1400.619142,14096.027173,0.654136,1.474520,163.032012,2,True,57
4,NeuralNetFastAI_BAG_L2,0.148595,0.148249,root_mean_squared_log_error,141.281441,1403.047288,14418.714421,1.918390,3.902666,485.719261,2,True,43
5,CatBoost_r9_BAG_L2,0.148619,0.148370,root_mean_squared_log_error,139.736524,1399.767245,14122.400661,0.373473,0.622623,189.405500,2,True,51
6,WeightedEnsemble_L2,0.148643,0.148106,root_mean_squared_log_error,10.848366,26.467676,5712.509628,0.009805,0.007650,17.217131,2,True,37
7,NeuralNetTorch_r79_BAG_L2,0.148651,0.148649,root_mean_squared_log_error,139.881446,1400.051847,14455.311912,0.518394,0.907225,522.316751,2,True,48
8,XGBoost_BAG_L2,0.148668,0.148653,root_mean_squared_log_error,139.833991,1400.443498,13974.733588,0.470939,1.298877,41.738427,2,True,44
9,LightGBMXT_BAG_L2,0.148677,0.148444,root_mean_squared_log_error,139.903392,1401.971957,14010.652983,0.540341,2.827335,77.657822,2,True,38


In [13]:
preds = predictor.predict(test)

INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.neighbors.KNeighborsRegressor.predict: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


In [14]:
submission = pd.DataFrame({'id': test.index, 'Rings': preds})
submission.to_csv(f'AutoGluon-{score:.6f}.csv', index=False)
submission.head()

,id,Rings
id,,
90615,90615,9.690414
90616,90616,9.642200
90617,90617,9.886904
90618,90618,10.299841
90619,90619,7.575377
